#### Table Doubling
Refers to the technique of doubling the table size (m) when the number of elements (n) becomes greater than m. So m -> 2m

Similarly, if m==(n/4), then shrink m by half i.e. m->m/2 

After every shrinkage or expansion, rehash all elements

Leads to an amortized(averaged) $\theta$(1) running time for k insertions and deletions

In [ ]:
import random

In [ ]:
class Node():
    def __init__(self, key=None, value=None):
        self.key=key
        self.value=value
        self.next=None

#### Large Prime Number Generation
The Dictionary class uses a large prime number generator. It is taken from the code at [this link](https://medium.com/@prudywsh/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb)

In [ ]:
from random import randrange, getrandbits
def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # Run k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p

def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 64):
        p = generate_prime_candidate(length)
    return p

#### Augmentations:
This is similar to the Dictionary class from Lecture 8, but this balances the hash table as we add and delete elements from it. This is done by calling the ```load_balance()``` function whenever the load_factor exceeds 1 or becomes smaller than 0.25

In [ ]:
class Dictionary():
    
    def __init__(self, m=8):
        from random import randint
        self.minsize=8
        self.m=m
        self.data=[None for _ in range(self.m)]
        self.p=generate_prime_number(length=256)
        self.a, self.b= randint(0,self.p-1), randint(0,self.p-1)
        self._length_=0
        self._alpha_=0
        self.keys=set()
    
    
    def universal_hashing_function(self, k):
        '''
        Implements the universal hashing function
        h(k)=[(ak+b) % p] % m
        k is the key to be hashed
        p is a large prime > U (size of the universe of keys)
        a,b are random non-negative integers smaller than p
        m is the size of the table
        '''
        k=hash(k)
        return ((self.a*k + self.b) % self.p) % self.m
    
    def insert(self, key, value, verbose=0):
        key_hash=self.universal_hashing_function(key)
        key_collision=0
        chain=0
        if not self.data[key_hash]:
            self.data[key_hash]=Node(key, value)
            self._length_+=1
        else:
            temp_node=self.data[key_hash]
            prev=None
            while(temp_node):
                if temp_node.key==key:
                    temp_node.value=value
                    key_collision=1
                    break
                else:
                    prev=temp_node
                    temp_node=temp_node.next
            if not key_collision:
                prev.next=Node(key, value)
                self._length_+=1
                chain=1
        self.keys.add(key)
        if verbose:
            print("{{{}:{}}} inserted".format(key, value))
        self._alpha_= self._length_/self.m
        if self._alpha_>1:
            print("Load factor exceeded 1. Dictionary length: {}\nGrowing table to {}...".format(self._length_, 2*self.m))
            self.load_balance(2*self.m)
        return chain
    
    def get_all_items(self)-> list:
        items=list()
        for key in self.keys:
            items.append([key,self.getitem(key)])
        return items
    
    def load_balance(self, new_m: int):
        items=self.get_all_items()
        self.m=new_m
        self.data=[None for _ in range(self.m)]
        self._length_=0
        for item in items:
            self.insert(item[0], item[1], verbose=0)
        self._length_=len(items)
        self._alpha_=self._length_/self.m
        print("Table balanced\nNew load factor: {}\nNew length: {}".format(self._alpha_, self._length_))
        
    
    def getitem(self, key):
        key_hash=self.universal_hashing_function(key)
        if not self.data[key_hash]:
            raise KeyError("Key {} does not exist".format(key))
        else:
            temp_node=self.data[key_hash]
            while(temp_node):
                if temp_node.key==key:
                    return temp_node.value
                else:
                    temp_node=temp_node.next
            raise KeyError("Key {} does not exist".format(key))

    def delete(self, key, verbose=0):
        key_hash=self.universal_hashing_function(key)
        if not self.data[key_hash]:
            raise KeyError("Key {} does not exist".format(key))
        else:
            temp_node=self.data[key_hash]
            prev=None
            while(temp_node):
                if temp_node.key==key:
                    if verbose:
                        print("Deleted key({}) with value ({})".format(temp_node.key, temp_node.value))
                    if not prev:
                        self.data[key_hash]=temp_node.next
                    else:
                        prev.next=temp_node.next
                    self._length_-=1
                    self._alpha_=self._length_/self.m
                    self.keys.remove(key)
                    if self._alpha_<0.25 and self.m>=4*self.minsize:
                        print("Alpha lower than 0.25. Dictionary length: {}\nShrinking table to {}".format(self._length_, self.m//4))
                        self.load_balance(self.m//4)
                    return
                else:
                    prev=temp_node
                    temp_node=temp_node.next
                    
            raise KeyError("Key {} does not exist".format(key))

    def show(self):
        for idx in range(len(self.data)):
            entry=self.data[idx]
            if entry:
                temp=entry
                while(temp):
                    print("{{ {} : {} }}".format(temp.key, temp.value), end=", ")
                    temp=temp.next

#### Testing code

In [ ]:
dictionary=Dictionary()
collisions=0
for i in range(64):
    collisions+=dictionary.insert(key=i, value=random.random())
print ("{} collisions".format(collisions))
dictionary.show()
for key in range(50):
    dictionary.delete(key, verbose=1)
dictionary.show()

#### String Matching
Given two strings s<sub>1</sub> and s<sub>2</sub>, does s<sub>1</sub> occur as a substring of s<sub>2</sub>?

**Naive method**: Check every substring of length |s<sub>1</sub>| 


In [ ]:
def simple_string_matching(s1, s2):
    return any(s1==s2[i:i+len(s1)] for i in range(len(s2)-len(s1)+1))

#### Karp-Rabin

This algorithm treats the string as a number. As a result, we can compute rolling hashes for the string by adding and removing characters.

**Rolling Hashes**: 
Given a rolling hash value *r* for a string *x*, we want to support the following operations:
- ```r.append(c)``` - Append the char c to end of x
- ```r.skip(c)``` - Remove first letter from x, assuming it is c

In [ ]:
class RollingHash():
    '''
    This class defines the Rolling Hash ADT
    It initializes values required to compute the hash 
    '''
    def __init__(self, lenx, base, p):
        self.umodp=0
        self.base=base
        self.p=p
        self.h=(self.base**(lenx-1))%p
    
    def append(self, c: str):
        '''
        This function computes the hash after adding 
        a character to the end of the string
        '''
        self.umodp=(self.umodp*self.base + ord(c))%self.p
    
    def skip(self, c: str):
        '''
        This function computes the hash after removing
        the first character from the string
        '''
        self.umodp=(self.umodp - self.h*ord(c))%self.p

In [ ]:
import math, random, tqdm
def Karp_Rabin(pattern: str, text: str, base: int, verbose=0):
    '''
    Does string matching in O(len(text)) time
    base: the number of characters in the system (256 for ASCII)
    verbose: set to 1 to see hashes when they match
    '''
    p=generate_prime_number(length=random.randint(128, 256))
    if verbose:
        print("Prime: {}".format(p))
    starts=[]
    pat_hash=RollingHash(len(pattern), base, p)
    text_hash=RollingHash(len(pattern), base, p)
    #generate hash for pattern and same number of letters in the text
    for i in range(len(pattern)):
        pat_hash.append(pattern[i])
        text_hash.append(text[i])
    if verbose:
        print("Pattern hash: {}\nText hash: {}".format(pat_hash.umodp, text_hash.umodp))
    if pat_hash.umodp==text_hash.umodp:
        if verbose:
            print("Hash matched")
        if pattern==text[:len(pattern)]:
            if verbose:
                print("Pattern found at 0")
            starts.append(0)
    for i in range(len(text)-len(pattern)):
        text_hash.skip(text[i])
        text_hash.append(text[i+len(pattern)])
        if verbose:
            print("Text hash: {}".format(text_hash.umodp))
        if pat_hash.umodp==text_hash.umodp:
            if verbose:
                print("Hash matched")
            if pattern==text[i+1:i+len(pattern)+1]:
                if verbose:
                    print("Pattern found at {}".format(i+1))
                starts.append(i+1)
    return starts

#### Testing code

In [ ]:
pattern="abc"
text="bdafvufvjavadv abcbakhvadfgvadvadogavafvkavavgasoufygfguyfvdvagabc"
base=256
Karp_Rabin(pattern, text, base, verbose=0)